In [67]:
import numpy as np
import pandas as pd

In [68]:
filepath = "data/2021_NCVR_Panse_001/dataset_ncvr_dirty.csv"
col_names = "sourceID,globalID,localID,FIRSTNAME,MIDDLENAME,LASTNAME,YEAROFBIRTH,PLACEOFBIRTH,COUNTRY,CITY,PLZ,STREET,GENDER,ETHNIC,RACE".split(",")
df = pd.read_csv(filepath, names=col_names, dtype={"PLZ": str, "YEAROFBIRTH": int},
                              keep_default_na=False)
df.shape[0]

200000

In [69]:
df1 = df[df.sourceID == "A"]
df2 = df[df.sourceID == "B"]
df1.shape[0]

100000

In [70]:
attr_names = "FIRSTNAME,MIDDLENAME,LASTNAME,YEAROFBIRTH,PLACEOFBIRTH,COUNTRY,CITY,PLZ,STREET,GENDER,ETHNIC,RACE".split(",")
def equal(row):
    for attr in attr_names:
        if row[attr + "_x"] != row[attr + "_y"]:
            return False
    return True

df_merged = pd.merge(df1, df2, how='inner', on='globalID')
df_merged[~df_merged.apply(equal, axis=1)]

,sourceID_x,globalID,localID_x,FIRSTNAME_x,MIDDLENAME_x,LASTNAME_x,YEAROFBIRTH_x,PLACEOFBIRTH_x,COUNTRY_x,CITY_x,...,LASTNAME_y,YEAROFBIRTH_y,PLACEOFBIRTH_y,COUNTRY_y,CITY_y,PLZ_y,STREET_y,GENDER_y,ETHNIC_y,RACE_y
0,A,AL82294,202338d686d641a11c6bae62644bd51e,LISA,F,BALDWIN,1966,,BUNCOMBE,BLACK MOUNTAIN,...,BALDWIN,1966,,BUNCOMBE,BLACK MOUNTAIN,28711,GENESIS,F,NL,W
1,A,BL352807,18471fd8e38e1ab6bd8d1545c368faa3,AHRMAD,RASHAD,GRIFFIN,1976,,DURHAM,DURHAM,...,GRIFFIN,1976,MO,DURHAM,DURHAM,27703,HOLLOWAY,M,UN,B
2,A,EA43706,180c3dfaab0583c4b23f137d48ab68ba,WHITNEY,MARIE,RAMER,1990,NC,STOKES,WALNUT COVE,...,CZERKIEWICZ,1990,NC,STOKES,WALNUT COVE,27052,PINK SMITH,F,NL,W
3,A,DT90164,58184c1d0360abfee773cc12a6ea76d4,WILLIAM,MARK,KING,1966,,ROWAN,SALISBURY,...,KING,1966,NC,ROWAN,SALISBURY,28147,GODBEY,M,NL,W
4,A,EH460595,eca1f5379f5282c6a63ee6adcacdacf2,MICHAEL,J,MARCHI,1963,CA,WAKE,WAKE FOREST,...,MARCHI,1963,CA,WAKE,WAKE FOREST,27587,OATLANDS,M,UN,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,A,AN131900,5e1fd8d90515d82f85724c893db7781b,MARK,EDWARD,LANE,1961,CT,CABARRUS,CONCORD,...,LANE,1961,,ROWAN,SALISBURY,28147,RANDOM,M,NL,W
19996,A,EH114155,1efbe3b2772db4e8f32d2b0161d32c0d,RUSSELL,T,GOLDNER,1961,WV,WAKE,RALEIGH,...,GOLDNER,1961,WV,LINCOLN,MAIDEN,28650,ANDERSON MOUNTAIN,M,NL,W
19997,A,EM28896,de30a5506ecb422cc08e16d02b36bcf0,JASPER,GLENN,SASSER,1942,,WAYNE,GOLDSBORO,...,SASSER,1942,NC,JOHNSTON,PRINCETON,27569,BLACK WATER,M,NL,W
19998,A,BN411997,d196b80860c62066eadaa21088856af5,DEBORAH,LOUSIE,BARKER,1970,LA,FORSYTH,KERNERSVILLE,...,JONES,1970,LA,FORSYTH,WALKERTOWN,27051,LA CREST,F,NL,W


### upsampling

In [71]:
from imblearn.over_sampling import SMOTENC
from collections import Counter

In [72]:
def dummy_df(shape, columns, dummy_value="DUMMY"):
    data = []
    for row_n in range(shape[0]):
        row = []
        for col_n in range(shape[1]):
            row.append(0.0 if col_n == 3 else dummy_value)
        data.append(row)
    return pd.DataFrame(data, columns=columns)

In [74]:
attr_names = "FIRSTNAME,MIDDLENAME,LASTNAME,YEAROFBIRTH,PLACEOFBIRTH,COUNTRY,CITY,PLZ,STREET,GENDER,ETHNIC,RACE".split(",")
desired_size = 1000
X = df[df.GENDER == "M"][:800]
X = X.drop(X.columns[[0, 1, 2]], axis=1)  # drop ID columns
sm = SMOTENC(random_state=42, categorical_features=[0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11])  # all but 3 (year of birth)
y = np.append(np.zeros(X.shape[0]), np.ones(desired_size))
X = pd.concat([X, dummy_df(shape=(desired_size, X.shape[1]), columns=X.columns)])
X_res, y_res = sm.fit_resample(X, y)
print(f'Resampled dataset samples per class {Counter(y_res)}')

Resampled dataset samples per class Counter({0.0: 1000, 1.0: 1000})


In [62]:
# remove dummy records (rows with class 1)
mask = [not bool(x) for x in y_res.tolist()]
X_res = X_res[mask]

In [63]:
# get all from X_res that are not in X
X_res[~X_res.apply(tuple,1).isin(X.apply(tuple,1))]

,FIRSTNAME,MIDDLENAME,LASTNAME,YEAROFBIRTH,PLACEOFBIRTH,COUNTRY,CITY,PLZ,STREET,GENDER,ETHNIC,RACE
1800,MAX,,JAFFA,1929.426111,,MECKLENBURG,CHARLOTTE,28205,WENSLEY,M,NL,W
1801,DALTHARD,,CURRIER,1930.816378,NC,HAYWOOD,WAYNESVILLE,28786,SUNNY HILL,M,NL,W
1802,ANGELA,LEE,MOULDEN,1970.000000,NC,GUILFORD,ELIZABETH CITY,,NC HWY 210,M,NL,B
1803,OLIVER,LANDON,HARVEY,1990.188691,NC,ALAMANCE,BURLINGTON,27217,JAMES,M,NL,W
1804,TIMOTHY,RAY,VOIGHT,1962.395583,NC,ORANGE,CHAPEL HILL,27253,NORTH CREEK,M,NL,W
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,THOMAS,DAVID,NAIL,1985.099200,FL,POLK,CLEMMONS,28756,HAWTHORNE,M,NL,W
1996,CHRISTOPHER,JOHN,ELLIOTT,1971.243237,NC,RANDOLPH,SALISBURY,27317,SCHOOL HOUSE,M,NL,W
1997,EDWIN,THOMAS,STUART,1959.341879,NC,PERSON,ROXBORO,28394,MEDFORD OAKLEY,M,NL,W
1998,ANGELA,LEE,LIVINGSTON,1971.684046,NC,GUILFORD,GREENSBORO,27263,RURAL RTE 6,M,NL,B


In [65]:
X_res[X_res.apply(tuple,1).isin(X.apply(tuple,1))]

,FIRSTNAME,MIDDLENAME,LASTNAME,YEAROFBIRTH,PLACEOFBIRTH,COUNTRY,CITY,PLZ,STREET,GENDER,ETHNIC,RACE
0,RONALD,EUGENE,LATTIMER,1953.0,OH,MOORE,PINEHURST,28374,BURNING TREE,M,NL,W
1,KURT,,BALEN,1968.0,PA,WAKE,APEX,27502,ASHLEY DOWNS,M,NL,W
2,WORTH,BROWN,FOUTZ,1919.0,,ROWAN,SALISBURY,28146,MORRISON,M,NL,W
3,LEOWARD,L,LOGAN,1961.0,NC,MECKLENBURG,CHARLOTTE,28215,DELGANY,M,NL,W
4,NGOWARI,,HARRY,1980.0,PA,MECKLENBURG,CHARLOTTE,28207,CRANBROOK,M,NL,B
...,...,...,...,...,...,...,...,...,...,...,...,...
795,RAEFORD,WILLARD,BYERLEY,1924.0,NC,CALDWELL,LENOIR,28645,OLD MORGANTON,M,NL,W
796,LELAND,DUANE,JACKSON,1964.0,NC,MOORE,CAMERON,28326,PLANK,M,NL,W
797,MICHAEL,A,JONES,1964.0,,MECKLENBURG,CHARLOTTE,28212,FARM POND,M,NL,B
798,BILLY,TED,RECTOR,1920.0,NC,HAYWOOD,CLYDE,28721,SUNNY HILL,M,NL,W


In [66]:
df.columns

Index(['sourceID', 'globalID', 'localID', 'FIRSTNAME', 'MIDDLENAME',
       'LASTNAME', 'YEAROFBIRTH', 'PLACEOFBIRTH', 'COUNTRY', 'CITY', 'PLZ',
       'STREET', 'GENDER', 'ETHNIC', 'RACE'],
      dtype='object')